In [ ]:
import os,sys,json
import kfp

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-limits-pl',
    description='pl with limits and requests '
)
def limits_requests_pipeline(token,limits_cpu,limits_memory,requests_cpu,requests_memory):

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="dkube-examples", run_script="python mnist/train.py",
                                    datasets='["mnist"]', outputs='["mnist"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1","LIMITS.cpu": "limits_cpu","LIMITS.memory": "limits_memory" ,"REQUESTS.cpu": "requests_cpu","REQUESTS.memory":"requests_memory"}]')

In [ ]:
client.create_run_from_pipeline_func(limits_requests_pipeline, arguments={"token":token,"limits_cpu": "500m","limits_memory": "128Mi", "requests_cpu": "250m","requests_memory": "64Mi"})